In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#the number of classes
no_of_classes = 5

In [7]:
#define sigmoid function:
def softmax(z):
    activ = np.exp(z)/np.sum(np.exp(z), axis = 0, keepdims = True)
    return activ

In [8]:
def cost_function_cross_entropy(softmax, Y):
    return Y.T*np.log(softmax)

In [19]:
def gradient(w,b,X,Y):
    
    m = X.shape[1]
    loss = 0.0;
    i = 0
    softmax_activ = softmax(np.dot(w,X) + b)
    for row in softmax_activ.T:
        loss += cost_function_cross_entropy(row.reshape(no_of_classes,1), Y[:, i])
        i+= 1
    loss = loss/m
    dw = np.dot(X, (softmax - Y).T)/m
    db = np.sum(logit - Y , axis = 1, keepdims = True)/m
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    
    gradient = {"dw" : dw, "db" : db}
    return gradient, loss
    

[[1]
 [2]
 [3]]
[[ 2]
 [34]
 [ 4]]


In [20]:
#optimize:
def optimization(w,b,X,Y, epoches, lr_alpha):
    costs = []
    for i in range(epoches):
        gradients, cost = gradient(w,b,X,Y)
        
        dw = gradients["dw"]
        db = gradients["db"]
        
        w = w - lr_alpha*dw
        b = b - lr_alpha*db
        if i % 100 == 0:
            costs.append(cost)
        params = {"w" : w, "b" : b}
    return params, costs

In [26]:
def prediction(w,b,X):
    m = X.shape[1] #no of data points
    Y_prediction = np.zeros((no_of_classes, m))#prediciton will be one hot encoding for each data
    w = w.reshape(no_of_classes, X.shape[0])
    b = b.reshape(no_of_classes, 1)
    soft = softmax(np.dot(w, X) + b)
    #print(logit)
    for i in range(soft.shape[1]):
        index = np.argmax(soft[:,i].reshape((1, no_of_classes)))
        Y_prediction[index, i] = 1
    assert(Y_prediction.shape ==(no_of_classes, m))
    return Y_prediction

In [25]:
x = np.array([[1,3,55,23,2],[1,2,44,55,2]])
for i in range(x.shape[1]):
    print(np.argmax(x[:, i].reshape(1,2)))

0
0
0
1
0
